# Face Detection

### 1. Using Haar Cascades in OpenCV

Haar cascades are a machine learning-based object detection method that predates the widespread use of deep learning. They are a cascade of classifiers that use a series of simple image features to identify objects or regions of interest in images.

In [13]:
import cv2
import time

# Load the pre-trained Haar cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Initialize the camera (assuming Raspberry Pi camera module)
cap = cv2.VideoCapture(0)

# Variables for FPS calculation
fps_start_time = time.time()
fps_frame_count = 0

while True:
    ret, frame = cap.read()
    
    # Convert the frame to grayscale (Haar cascades work on grayscale images)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Perform face detection
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    # Draw rectangles around the detected faces
    for (x, y, w, h) in faces:
        # Draw rectangle, bounding box. (x,y) is the upper left edge of box
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

        # Draw dot at center of bounding box (x + w//2, y + h//2)
        cv2.circle(frame, (x + w//2, y + h//2), 3, (0, 255, 0), -1)  # Green dot

        # Display coordinates at the top of rectangle
        cv2.putText(frame, f'x: {x}, y: {y}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # Calculate FPS
    fps_frame_count += 1
    if fps_frame_count >= 1:
        fps_end_time = time.time()
        fps = fps_frame_count / (fps_end_time - fps_start_time)
        fps_frame_count = 0
        fps_start_time = fps_end_time
        
        # Print FPS
        # print("FPS: {:.2f}".format(fps))
        cv2.putText(frame, "FPS: {:.2f}".format(fps), (10, 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    
    # Display the frame with detected faces
    cv2.imshow('Face Detection', frame)
    
    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close the OpenCV window
cap.release()
cv2.destroyAllWindows()

On CPU it is giving over 20 fps.

### 2. Using cvlib's face_detect 

The `cvlib.detect_face` function in the `cvlib` library uses a pre-trained deep learning model for face detection. The model is based on a Single Shot Multibox Detector (SSD), which is a popular architecture for object detection tasks. It uses a pre-trained caffe model, with a modified res-net backbone.

In [13]:
import cv2
import cvlib
import time

# Initialize the camera (assuming default camera)
cap = cv2.VideoCapture(0)

# Variables for FPS calculation
fps_start_time = time.time()
fps_frame_count = 0

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Perform face detection
    faces, confidences = cvlib.detect_face(frame)

    # print(faces.type)

    # Loop over detected faces and draw rectangles
    for face in faces:
        (startX, startY, endX, endY) = face
        cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)

        # # Draw dot at center of bounding box (x + w//2, y + h//2)
        # coordinateX = (startX + endX)//2
        # coordinateY = (startY + endY)//2
        # cv2.circle(frame, (coordinateX, coordinateY), 3, (255, 255, 255), -1)  # White dot
        # # print(f"x: {coordinateX}, y: {coordinateY}")
    
        # Draw dot at the neck, that is below of bounding box
        # neck position
        coordinateX = (startX + endX) // 2
        coordinateY = endY + int(0.25*(endY - startY))
        cv2.circle(frame, (coordinateX, coordinateY), 3, (0, 255, 0), -1)  # Green dot

    # Calculate FPS
    fps_frame_count += 1
    if fps_frame_count >= 1:
        fps_end_time = time.time()
        fps = fps_frame_count / (fps_end_time - fps_start_time)
        fps_frame_count = 0
        fps_start_time = fps_end_time

        # Print FPS
        cv2.putText(frame, "FPS: {:.2f}".format(fps), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    # Display the frame with detected faces and FPS
    cv2.imshow('Face Detection', frame)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close the OpenCV window
cap.release()
cv2.destroyAllWindows()

On CPU giving over 20 FPS, and can detect faces better.